<a href="https://colab.research.google.com/github/doremococo/python-bootcamp/blob/main/chapter3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np


df = pd.read_csv("/content/adult_income_dataset.csv")

# 列名を取得
print(df.columns)

# 3-01 特定列のユニークな行を出力
df["occupation"].unique()

# ユニークな値がいくつあるか数える
df["occupation"].nunique()

# その列の各値がどれくらいの数出現しているかを確認 .value_counts()
df["workclass"].value_counts()

# 3-02 値の置換 .replace("検索文字", "置換後の文字")
new_df = df.copy()
unique_education = df["education"].value_counts()
print(f"old:  {unique_education}")

new_df["education"] = new_df["education"].replace("Bachelors", "Bachelor degree")
# new_unique_education = new_df["education"].value_counts()
# print(f"new:  {new_unique_education}")

# 色々な置換方法を試す
# replace(dict)
dict_rpls = new_df[["capital-loss", "capital-gain"]].replace({0: 1111, 2174: 9999})
# print(dict_rpls)


Index(['age', 'workclass', 'fnlwgt', 'education', 'education-num',
       'marital-status', 'occupation', 'relationship', 'race', 'sex',
       'capital-gain', 'capital-loss', 'hours-per-week', 'native-country',
       'income'],
      dtype='object')
old:  education
HS-grad         10501
Some-college     7291
Bachelors        5355
Masters          1723
Assoc-voc        1382
11th             1175
Assoc-acdm       1067
10th              933
7th-8th           646
Prof-school       576
9th               514
12th              433
Doctorate         413
5th-6th           333
1st-4th           168
Preschool          51
Name: count, dtype: int64


In [2]:
new_df = df.copy()
new_df["capital-diff"] = df["capital-gain"] - df["capital-loss"]
new_df[["capital-gain", "capital-loss", "capital-diff"]].head()

,capital-gain,capital-loss,capital-diff
0,2174,0,2174
1,0,0,0
2,0,0,0
3,0,0,0
4,0,0,0


In [3]:
from sklearn.preprocessing import StandardScaler

df_304 = df.copy()


# 3-04 列の標準化
scaler = StandardScaler()
df_304[["scl-age", "scl-cg"]] = scaler.fit_transform(df_304[["age","capital-gain"]])
print(df_304["age"].mean())
df_304[["scl-age", "age"]].head()

38.58164675532078


,scl-age,age
0,0.030671,39
1,0.837109,50
2,-0.042642,38
3,1.057047,53
4,-0.775768,28


In [4]:
# 3-05 外れ値を検出する
# new_df の hour-per-week列で外れ値を四分位範囲で検出し、その数を出力
# 四分位範囲(IQR) = 3Q - 1Q

new_df = df.copy()

Q1 = new_df["hours-per-week"].quantile(0.25)
Q3 = new_df["hours-per-week"].quantile(0.75)

IQR = Q3 - Q1
print(IQR)

# 経験則として、1.5*IQR が外れ値までの距離
outer_Q1 = Q1 - 1.5*IQR
outer_Q3 = Q3 + 1.5*IQR

pre_count_sum = ((new_df["hours-per-week"] > outer_Q1) & (new_df["hours-per-week"] < outer_Q3)).sum()
print("pre:  ", pre_count_sum)

# 外れ値の数をかぞえる
count_outlnier = ((new_df["hours-per-week"] < outer_Q1) | (new_df["hours-per-week"] > outer_Q3)).sum()
print("pre_out:  ", count_outlnier)

# 外れ値にある値を、Q1もしくはQ3に置換する
new_df["hours-per-week"] = np.where(new_df["hours-per-week"] < outer_Q1 , Q1, new_df["hours-per-week"])
new_df["hours-per-week"] = np.where(new_df["hours-per-week"] > outer_Q3 , Q3, new_df["hours-per-week"])

# 外れ値の数をかぞえる
count_outliner = ((new_df["hours-per-week"] < outer_Q1) | (new_df["hours-per-week"] > outer_Q3)).sum()
print("fix_out:  ", count_outliner)

new_df["hours-per-week"].head()

# それをさらに、pandas の clip によって置換する
outer_Q1 = 45
outer_Q3 = 46
new_df["hours-per-week"] = new_df["hours-per-week"].clip(lower = outer_Q1, upper = outer_Q3)
new_df["hours-per-week"].head(10)


5.0
pre:   23553
pre_out:   9008
fix_out:   0


,hours-per-week
0,45.0
1,45.0
2,45.0
3,45.0
4,45.0
5,45.0
6,45.0
7,45.0
8,46.0
9,45.0


In [5]:
# 3-06 欠損値を特定の値で埋める
new_df = df.copy()

# occupation 列の欠損値の数を出力
print("pre null:  ", new_df["occupation"].isnull().sum())

# occupation列の欠損値を特定の値で埋める

# 試しに np.where でやっても出来た
new_df["occupation"] = np.where(new_df["occupation"].isnull(), "AAAAA", new_df["occupation"])
print("test np null -> AAAAA:  ", (new_df["occupation"] == "AAAAA").sum())
new_df["occupation"] = np.where(new_df["occupation"] == "AAAAA", np.nan , new_df["occupation"])

# 基本的には pandas の fillna("") で欠損値を置換する
new_df["occupation"] = new_df["occupation"].fillna("BBBBB")
print("fillna(BBBBB):  ", (new_df["occupation"] == "BBBBB").sum())

pre null:   1843
test np null -> AAAAA:   1843
fillna(BBBBB):   1843


In [6]:
# 3-07 データの正規化
# 0-1の間で正規化を行う。対象["hours-per-week"]
new_df = df.copy()

from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
new_df["nom-hpw"] = scaler.fit_transform(new_df[["hours-per-week"]])
new_df[["nom-hpw", "hours-per-week"]].head()

,nom-hpw,hours-per-week
0,0.397959,40
1,0.122449,13
2,0.397959,40
3,0.397959,40
4,0.397959,40


In [7]:
# 3-08 特定の行の"インデックス"を抽出する loc | iloc で実装
df.iloc[4:8]
df.iloc[[1, 10]]

# 3-09 特定の行の"行の名前"から取得する
print("loc[3]:  ", df.loc[4])
print("iloc[3]:  ", df.iloc[4])


loc[3]:   age                               28
workclass                    Private
fnlwgt                        338409
education                  Bachelors
education-num                     13
marital-status    Married-civ-spouse
occupation            Prof-specialty
relationship                    Wife
race                           Black
sex                           Female
capital-gain                       0
capital-loss                       0
hours-per-week                    40
native-country                  Cuba
income                         <=50K
Name: 4, dtype: object
iloc[3]:   age                               28
workclass                    Private
fnlwgt                        338409
education                  Bachelors
education-num                     13
marital-status    Married-civ-spouse
occupation            Prof-specialty
relationship                    Wife
race                           Black
sex                           Female
capital-gain                   

In [8]:
# 3-10 行の名前が3, 5, 7 カラムがage, workclass, education, hours-per-week のものを抽出
selected_df = df.loc[[3, 5, 7], ["age", "workclass", "education", "hours-per-week"]]

In [9]:
# 3-11 列の値に特定の文字を結合し、列に反映
new_df = df.copy()
new_df["prefix_native_country"] = "Country_" + new_df["native-country"]
new_df["prefix_native_country"].head()

,prefix_native_country
0,Country_United-States
1,Country_United-States
2,Country_United-States
3,Country_United-States
4,Country_Cuba


In [10]:
# 3-11 新しい数値で更新したい。数値系のカラムに対して、計算結果を反映させたい
# hours-per-week の数値を7で割って hours-per-day 列を作成
new_df = df.copy()
new_df["hours-per-day"] = new_df["hours-per-week"] / 7
new_df[["hours-per-week", "hours-per-day"]].head()

,hours-per-week,hours-per-day
0,40,5.714286
1,13,1.857143
2,40,5.714286
3,40,5.714286
4,40,5.714286


In [11]:
# 3-13 特定の条件にマッチしたレコードだけ更新する
# education-num >= 12 の時 は High, それ以外の時は Low と出力する education-level 列を作成
# education-num , education-level 列を並べて表示
new_df = df.copy()

# まずは 正攻法の lambda + apply で実装
%timeit new_df["app-edu-level"] = new_df["education-num"].apply(lambda x: "High" if x >= 12 else "Low")

# np.where を使って置換。こっちの方が圧倒的に高速
%timeit new_df["np-edu-level"] = np.where(new_df["education-num"] >= 12, "High", "Low")

new_df[["education-num", "app-edu-level", "np-edu-level"]]

7.38 ms ± 2.48 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)
2.67 ms ± 111 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


,education-num,app-edu-level,np-edu-level
0,13,High,High
1,13,High,High
2,9,Low,Low
3,7,Low,Low
4,13,High,High
...,...,...,...
32556,12,High,High
32557,9,Low,Low
32558,9,Low,Low
32559,9,Low,Low


In [12]:
# 3-14 複数の条件で列の値を更新する
# 複数列を跨いだ条件に応じた出力をした status 列を作る
# income > 50k & hours-per-week >= 40 ==>> High income full-time
# 上記以外 ==>> other

new_df = df.copy()

# 先ほどの np.where でやってみる
cond = ((new_df["income"] == ">50K") & (new_df["hours-per-week"] >= 40))
new_df["status"] = np.where(cond, "High", "other")
new_df[["income", "hours-per-week", "status"]].head(10)

,income,hours-per-week,status
0,<=50K,40,other
1,<=50K,13,other
2,<=50K,40,other
3,<=50K,40,other
4,<=50K,40,other
5,<=50K,40,other
6,<=50K,16,other
7,>50K,45,High
8,>50K,50,High
9,>50K,40,High


In [35]:
# 3-16 特定の値で分割し、別カラムに移す
# education で - を判定し分割 example: "High-edu" -> "High", "edu"
new_df = df.copy()
new_df[["education-level", "education-type"]] = new_df["education"].str.split("-", expand=True)
new_df[["education", "education-level", "education-type"]].head()

,education,education-level,education-type
0,Bachelors,Bachelors,None
1,Bachelors,Bachelors,None
2,HS-grad,HS,grad
3,11th,11th,None
4,Bachelors,Bachelors,None
